In [6]:
# desinging  nerual network

import numpy as np
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1= 300
n_hidden2 =100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [12]:
import keras

with tf.name_scope("dense"):
    hidden1 = dense(X,n_hidden1,name="hidden1", activation=tf.nn.relu)
    hidden2 = dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = dense(hidden2, n_outputs, name="outputs")

In [13]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar('log_loss', loss)


In [14]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer =tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [24]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy',accuracy)

In [17]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [18]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir,name)

In [19]:
logdir = log_dir("mnist_dnn")

In [20]:
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [26]:
# execturion

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")


n_epochs =20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch =  mnist.train.next_batch(batch_size)
            
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
        
        accuracy_val, loss_val, accuracy_summary_str, loss_summary_str = sess.run([accuracy, loss,accuracy_summary, loss_summary], feed_dict={X: X_batch, y: y_batch})
        file_writer.add_summary(accuracy_summary_str,epoch)
        file_writer.add_summary(loss_summary_str,epoch)
        print(epoch, "Train accuracy: ", acc_train, "Test accuracy: ", acc_test)
    
    save_path = saver.save(sess, "./my_model.final.ckpt")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0 Train accuracy:  0.92 Test accuracy:  0.8955
1 Train accuracy:  0.98 Test accuracy:  0.9173
2 Train accuracy:  0.96 Test accuracy:  0.928
3 Train accuracy:  0.92 Test accuracy:  0.9364
4 Train accuracy:  0.94 Test accuracy:  0.9403
5 Train accuracy:  0.98 Test accuracy:  0.9452
6 Train accuracy:  1.0 Test accuracy:  0.9484
7 Train accuracy:  0.98 Test accuracy:  0.9509
8 Train accuracy:  1.0 Test accuracy:  0.9536
9 Train accuracy:  0.96 Test accuracy:  0.9567
10 Train accuracy:  1.0 Test accuracy:  0.9591
11 Train accuracy:  0.98 Test accuracy:  0.9606
12 Train accuracy:  0.98 Test accuracy:  0.9629
13 Train accuracy:  0.98 Test accuracy:  0.9632
14 Train accuracy:  0.96 Test accuracy:  0.9643
15 Train accuracy:  1.0 Test accuracy:  0.9656
16 Train accuracy:  0.96 Test accuracy:  0.9679
17 Train 